Projet de Machine Learning : expliquer le prix de l'électricité

Romain Foucher
Matthieu Evort
Julie Chen


Introduction :

Le machine learning est une technologie qui vise à apprendre aux machines à tirer des enseignements des données et à s'améliorer avec l'expérience, au lieu d'être explicitement programmées pour le faire.

Ce projet consiste a expliquer le prix de l'électricité selon une multitude de facteurs. On peut citer le climat, le réchauffement climatique, les  événements géopolitiques... De plus, chaque pays s’appuie sur un mix  énergétique qui lui est propre (nucl ́eaire, solaire, hydrolique, gaz, charbon, etc).

 Pour cela, nous allons utiliser plusieurs méthodes afin de préparer nos données, de les analyser et de les modéliser grâce aux cours et aux TP que nous avons pu effectuer en cours mais ainsi qu'à la documantation d'internet.

Sommaire :

1) Préparation des données
2) Analyse exploratoire des données
3) Modélisation des données


1) Préparation des donnés

Afin de préparer nos données nous allons remplir les cases vides par la médianne de la colonne. En effet, si nous supprimons les lignes ou les colonnes, nous perdrons des informations et nous réduisons la taille de l'échantillon.

In [6]:
from Data_preparation import data_x

print(0.0 in data_x.values)



         ID  DAY_ID  COUNTRY  DE_CONSUMPTION  FR_CONSUMPTION  DE_FR_EXCHANGE   
0     False   False    False           False           False           False  \
1     False   False    False           False           False           False   
2     False   False    False           False           False           False   
3     False   False    False           False           False           False   
4     False   False    False           False           False           False   
...     ...     ...      ...             ...             ...             ...   
1489  False   False    False           False           False           False   
1490  False   False    False           False           False           False   
1491  False   False    False           False           False           False   
1492  False   False    False           False           False           False   
1493  False   False    False           False           False           False   

      FR_DE_EXCHANGE  DE_NET_EXPORT  FR

NameError: name 'df' is not defined